# Checkpoint 1: Neural networks and deep learning
---
*Responsible:* Guillermo Hamity (<ghamity@ed.ac.uk>)

In this checkpoint exercise, we will use neural networks to predict the **type** of weather *given* the available ground observations. You will be using observation data from **June 2019** across all UK Met Office weather stations.

### Notes on the Dataset
* You will be using weather observation data from the UK Met Office Datapoint service
* Ground observations are made hourly at weather stations across the length of the UK 
* The data sample covers data from June 2019
* Data collections for each day starts at 6.30pm. All observation data is listed in one day blocks
* The time value column refers to the number of minutes after midnight 
* `Null` values for some features are expected (e.g. Wind Gust)
* Data import and preparation is already provided 


This week, I am not providing example notebooks like `lecture2.ipynb` and `data-science-tools.ipynb` for Unit 2, though these may still be useful to you. Instead, I am **providing the imports for all of the modules and classes that you should need.** Think of these as LEGO blocks; you have the ones you need but may look up how to "assemble" them.

### Notes on assessment
* Try and calculate the answers to the exercises provided. If you are unable to complete the question, describe which approach you _would_ have taken to solve the problem
* Code must be understandable and reproducible. Before grading the notebook kernel **may** be restarted and re-run, so make sure that your code can run from start to finish without any (unintentional) errors
* If you are unsure on how to proceed please **ask one of the TAs** during the workshop
- Notebooks should be submitted by **10am on Friday 9 October 2021** 
- This CP exercise sheet is divided into **6 sections**, corresponding to parts of the lecture, giving a maximum of **10 marks** in total:

| <p align='left'> Title                         | <p align='left'> Exercise nos. | <p align='left'> Number of marks |
| ------------------------------------- | ----- | --- |
| <p align='left'> 1. Conceptual questions               | <p align='left'>  1–5  | <p align='left'> 2.5 |
| <p align='left'> 2. Data preprocessing and RandomForest                | <p align='left'>  6–9  | <p align='left'> 2.5 |
| <p align='left'> 3. Neural networks in `scikit-learn`  | <p align='left'>  10–11 | <p align='left'> 1.5 | 
| <p align='left'> 4. Neural networks in `Keras`         | <p align='left'> 12–13 | <p align='left'> 2 |
| <p align='left'> 5. Regularisation                     | <p align='left'> 14–15 | <p align='left'> 1.5 |
| <p align='left'> 6. Bonus: Hyperparameter optimisation | <p align='left'> 16 | <p align='left'> 1.0 (\*bonus\*) |
| <p align='left'> **Total** | | <p align='left'> **10 + 1** |

- The total number of marks allocated for this CP is 10,
    - 1 additional mark can be given (maximimally up to 10 marks in total) for "bonus" exercise on hyperparameter optimisation. If you are pressed for time, focus on the first five sections; those are the core ones.
    - Half marks may be deducted for code legibility (i.e. very difficult to tell what you are doing), or for badly formated plots (i.e. no legends, axis labels etc.). The TAs will use their discression for this so comment code when applicable and keep relevant information in your plots.

_Note:_ You can suppress double-printing of plots from the `plot` module by either _(a)_ adding a semicolon after the function call (_i.e._ `plot.<method>(...);`), or _(b)_ by capturing the return `pyplot.Figure` object as a variable (_i.e._ `fig = plot.<method>(...)`).

## Preamble

In [ ]:
# Standard import(s)
import numpy as np
import pandas as pd
import random as rn
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

# Suppress unnecessary ConvergenceWarnings and DeprecationWarnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

# Set a random seed variable to make workbook reproducible
seed=5
np.random.seed(seed)
rn.seed(seed)
os.environ['PYTHONHASHSEED']=str(seed)
tf.compat.v1.set_random_seed(seed)

# Switch off multi-threading for TensorFlow
from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                  inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
K.set_session(sess)

In [ ]:
# Load in the prepared weather data
obs = pd.read_csv('weather.csv')
obs.head()

In [ ]:
obs.shape

In [ ]:
obs.describe()

For this exercise we will use **8** input features (provided) and clean the data:

In [ ]:
# Define 8 input feature variables, 1 target variable data, and names of the 3 weather types
features = ['Latitude', 'Elevation', 'Temperature', 'Visibility', 'WindSpeed', 'Pressure', 'Humidity', 'WindDirection']
output   = ['Type']
wtype    = ['Clear', 'Cloudy', 'Precip']

Define derived dataset containing only the relevant columns and rows.

In [ ]:
# Reduce to feature and type columns
dataset = obs[features + output]

# Drop duplicates and null values 
dataset = dataset.drop_duplicates().dropna()

# Drop unrecorded weather type
dataset = dataset[dataset.Type != 3]

# Check shape 
dataset.shape

## 1. Conceptual questions (2.5 Marks)
---
This section covers **5** exercises on conceptual understanding of neural networks.

#### 1. Which are the most used activation functions and why do we (typically) need non-linear activation functions in neural networks? (0.5 mark)

Most used activation functions: 
* Sigmoid
* ReLU
* Softmax
* tanh

We typically need non-linear activation functions because their aim in a neural network is to produce a nonlinear decision boundary via non-linear combinations of the weight and inputs. Hidden layers become useless if we use linear activation functions because the composition of linear functions is itself a linear function. It is also not possible to use backpropagation to train a model using linear activation functions because the derivative of linear functions is a constant and has no relation to the input. This implies that it is not possible to go back and deduce which weights can provide a better prediction. Using non-linear models also allows the model to learn more complex functions.

#### 2. Why do we need deep neural networks and which are the main differences between deep and shallow learning? (0.5 Mark)

We need deep neural networks as they are able to:
* Exploit high-dimensional data
* Learn relevant features of the input and make predictions based on those characteristics
* Perform feature extraction (to find the best subset of features) and deep combination simultaneously

The main differences between deep and shallow learning are:

| Shallow Learning | Deep Learning | 
| :- | :- |
| Unable to exploit high-dimensional data | Able to exploit high-dimensional data  |
| Features are engineered manually | Network learns relevant features by itself |
| Combined using simple algorithms to get output | Feature extraction and Deep combination are done simultaneously to produce output |

#### 3. Discuss the Bias-variance trade-off and its relation to underfitting and overfitting of a model. Which are the caractheristics of an ideal model?  (0.5 mark)

The bias-variance trade-off is a fundamental principle for understanding the generalization of predictive learning models. The fitting of a model has to do with whether variance and bias are high or low. Overfitting is the result of a model with  low bias and high variance, while underfitting is a model with high bias and low variance. The worst model is a model with both variance and bias being high. The ideal model has low variance and low bias.

#### 4. Given a neural network with 4 input nodes, 2 layers with 5 nodes each, and 1 output node, what is the total number of free (trainable) parameters in the network? Does it matter which activation function(s) are used?  (0.5 mark)

The total number of trainable parameters in the network is $(4 \times 5) + ( 5 \times 5 ) + (5 \times 1 ) + ( 5 + 5 + 1 ) = 61$.

In general, different activation functions are suitable for specific problems:
* The **tanh** function is suitable for regressing, both positive and negative quantities
* The **ReLU** function is suitable for regressing, non-negative quantities
* The **Sigmoid** function is suitable for binary classification probability
* The **Softmax** function is suitable for multiclass classification probability

Hidden layers use the same activation function than output layers. As we saw above, the activation function for output layers depends on the type of prediction problem.

#### 5. What are appropriate choice for _(a)_ the number of output nodes and _(b)_ output activation function(s) for each of the following tasks, and why? (0.5 mark)

1. Regression of the $x$, $y$, and $z$ coordinates of a single particle in an arbitrary coordinate system
2. Regression of particle energy of a single particle
3. Classification of two processes (signal vs. background)
4. Classification among *N* classes (dog vs. cat vs. fish vs. ...)

1. **(a)** 3 (output is predicted $x,y,z$ coordinates, \
   **(b)** tanh (we need the function to take any real value as input)
2. **(a)** 1 (output is predicted energy of the particle which is a single value), \
   **(b)** ReLU (we have regression of non-negative quantities as energy can only be $>0$)
3. **(a)** 2 (output is the class of either of the two processes), \
**(b)** Sigmoid (binary classification probability as the output is the probability that each process is either signal or background)
4. **(a)** *N* (output is probability for each class), \
**(b)** Softmax (we have multiple classes -multiclass- and need to obtain probabilities of input belonging to each of these)

## 2. Data preprocessing and RTs (2.5 mark)
---
This section covers **4** exercises on data preparation, feature standardisation, and dataset splitting.

---
**_Comment on target format and one-hot encoding:_** By default, the target column (`Type`) contains one integer (0, 1, or 2) for each example, the integer specifying one of three possible types of weather. However, for doing multi-class classification (which this is), we want our neural network to have one output node per class (_i.e._ 3 output nodes in this case), such that the activation of each output node is interpreted as the likelihood for a given sample being of the type in question. Therefore, the target should also be a 3-element vector for each sample; this vector should be all zeros, except for a $1$ at the index corresponding to the type in question. This is called **one-hot encoding**, and a few examples are shown below:

- type = 0 $\to$ one-hot = $[1, 0, 0]$ for 3 classes
- type = 1 $\to$ one-hot = $[0, 1, 0]$ for 3 classes
- type = 2 $\to$ one-hot = $[0, 0, 1]$ for 3 classes

This is the target towards which a neural network classifier is trained: That is, ideally, for an example of type 0, the network will output a large activation ($\approx 1$) on the first output node (interpreted as a large likelihood for the first weather type), and very small activations ($\ll 1$) on the two other output nodes (intepreted as small likelihoods for the two other weather types); and so on.

The same type of one-hot encoding can be performed for any number of target classes $N_{c}$, which just results in $N_{c}$-element target vectors with a single non-zero entry each.

To be user friendly, however, `scikit-learn` allows us to use integer targets for multi-class classification — it does the one-hot encoding for us "under the hood." Similarly, `keras`, _can_ also allow us to use integer targets for multi-class classification, provided we use the appropropriate loss (`sparse_categorical_crossentropy`). Otherwise (if we use `categorical_crossentropy` loss), it expects one-hot encoded targets. Which approach you choose is up to you — but now you know what goes on.

---

In [ ]:
# Relevant import(s) for this section
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics # Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing # Import preprocessing for String-Int conversion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

#### 6. Prepare the feature and target arrays (0.5 mark)
- Randomly sample **3,500** observations per weather type (**10,500** observations in total) from `dataset` into a new `pandas.DataFrame`; call it `sample`.
- One-hot encode the **wind direction** variable (_i.e._ $N$ to $[1, 0, \ldots, 0]$, $NNE$ to $[0, 1, \ldots, 0]$, _etc._ ), to allow us to input it to the neural network. The exact order of the encoding (_i.e._ which direction corresponds to which index) doesn't matter. *Hint:*
  - *Either:* Use the scikit-learn `ColumnTransformer` with the `OneHotEncoder` applied to the `WindDirection` column, and let the remainder of the features pass through un-transformed.
  - *Or:* Use the `OneHotEncoder` class directly on the `WindDirection` column (use `sparse=False` in the `OneHotEncoder` constructor), and then concatenate with a `numpy.array` containing the remaining features.
- Define `numpy.arrays` named `X` and `y` containing the training features (the 7 unmodified ones plus the one-hot encoded wind directions) and target, respectively.
- Argue whether the shapes of `X` and `y` are as expected/as they should be.

In [ ]:
dataset

In [ ]:
# randomly sample 3,500 observations per weather type
sample = dataset.groupby("Type").apply(lambda x: x.sample(n=3500))
#sample = dataset.groupby("Type").sample(3500)

sample.shape # to verify we have correct sample size

In [ ]:
sample

In [ ]:
# use the scikit-learn ColumnTransformer with the OneHotEncoder applied to the WindDirection column, 
# and let the remainder of the features pass through un-transformed.
# columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(categories='auto', sparse=False), [7])], remainder='passthrough')

columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(categories='auto', sparse=False), ['WindDirection'])], remainder='passthrough')


# fit and transform the dataset to label encode and one hot encode the column
all_trained = np.array(columnTransformer.fit_transform(sample), dtype = float)

# training features should exclude Type as that is the target
X = all_trained[:,0:-1]
X.shape

In [ ]:
# printing the matrix to visualise the result

WindDirs = sample['WindDirection'].unique()
WindDirs.reshape(-1,1)
print(WindDirs)
print(OneHotEncoder(sparse=False).fit_transform(WindDirs.reshape(-1,1)))
print(WindDirs.shape)

In [ ]:
# target array is the weather type
y = all_trained[:,-1]
print(y.shape)

The arrays are related by $f(X) = y$. So $X$ represents the input to the function and $y$ is the output. Since the training features $X$ are 2-dimensional we expect that $y$ will be a one-dimensional array (a vector). So both shapes are as expected.

#### 7. Train a Random Forest and evaluate the performance (1 mark)

Decision trees work well with a mixture of features (of different scales, and bot binary and continuos), so we will train a random forest to do the job of categorisation.

You are given the train test split:

In [ ]:
#Import random fosets and confusion matrix metric
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
print(x_train.shape,x_test.shape)

1. Train a RandomForestClassifier with 1000 estimators, `gini` seperation criteria, and max depth 4.
2. Check the overal accuaracy on the testing set

In [ ]:
# train random forest
rf = RandomForestClassifier(n_estimators=1000, criterion = 'gini', max_depth=4)

In [ ]:
# check accuracy on the testing set
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print("Accuracy Testing:",metrics.accuracy_score(y_test, y_pred))

3. Use the confusion_matrix method to return the confusion matrix normalised over the true lables, i.e. sum over rows should sum to 100%. Use the given colormap to plot the confusion matrix in a heatmap.
    - Define the axis tick names to represent Clear, Cloudy or Precip
    - Use suitable x and y axis labels
    
4. What are the true positive rates for clear, cloudy and perp? What is the probability that rain is forcast given that the day is clear?
5. Which features does the random forest deem the most important?

In [ ]:
fig, ax = plt.subplots()
colormap = sns.diverging_palette(220, 10, as_cmap=True)
matrix = confusion_matrix(y_test, y_pred)

# normalise the matrix
row_sums = matrix.sum(axis=1)
new_matrix = matrix / row_sums[:, np.newaxis]

sns.heatmap(new_matrix, cmap='Blues', annot=True, xticklabels=wtype, yticklabels=wtype,fmt=".4f", square=True)
#ax.set_ylim(3,-0.5)
ax.set_ylabel('True Weather')
ax.set_xlabel('Predicted Weather')
plt.show()

4. a) What are the true positive rates for clear, cloudy and perp? What is the probability that rain is forcast given that the day is clear?

**True positive rates:**
* Clear: 71%
* Cloudy: 49%
* Precipitation: 80%

4. b) What is the probability that rain is forcast given that the day is clear?

$P(rain forecast| clear true weather) = \frac{P(clear true weather|rain forecast) \cap P(rain forecast)}{P(clear true weather)} $

where $P(clear true weather| rain forecast) = 0.0559$ 

so, $ P((rain forecast| clear true weather) = 0.0559$

In [ ]:
# define importances
importances = rf.feature_importances_

In [ ]:
##### Given plotting example for feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(23), rf.feature_importances_)
plt.yticks(range(23), np.concatenate([WindDirs,features[:-1]]))
plt.show()

From the bar chart it is apparent that the random forest deems Visibility, Pressure and Humidity to be the most import important features (in ascending order).

#### 8. Standardise the relevant features (0.5 mark)

_Note:_ You shouldn't standardise the one-hot encoded wind directions; they already have the desired format. Perform a sanity check to make sure that the resulting features have the expected distributional properties (mean and standard deviation; or minimum and maximum value).
- Hint:

    - Use the scikit-learn `StandardScaler`
    - Or use the scikit-learn `MinMaxScaler`

- Perform a sanity check to make sure that the resulting features have the expected distributional properties (mean and standard deviation; or minimum and maximum value).
    - The number of columns should match, and depending on the choice of standardisation, the last 7 columns should either have:
      - (Using `StandardScaler`) means = 0 and standard deviations = 1; or
      - (Using `MinMaxScaler`) min = 0, max = 1

In [ ]:
#col_names = sample.columns.values

scaler = StandardScaler()

# exclude onehotencoded features 

X[:,len(WindDirs):]= scaler.fit_transform(X[:,len(WindDirs):])

#### 9. Split the dataset into a training and a testing part (0.5)

Reserve **30%** of data for testing. Check whether the resulting arrays have the expected shapes.

In [ ]:
# split dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test
print(x_train.shape,x_test.shape)

Arrays have expected shapes

## 3. Neural networks in `scikit-learn` (1.5 mark)
---
This section covers **2** exercises on constructing and training neural networks using the `scikit-learn` library, as well as evaluating neural network performance. `scikit-learn` provide many, very easy to use ML algorithms, including neural networks. These are called `MLPClassifier` (MLP = multi-layer perceptron; a historic name for densely connected, feed-forward neural networks) when used for classification, and `MLPRegressor` when used for regression. We will focus on the former for now.

In [ ]:
# Relevant import(s) for this section
from sklearn.neural_network import MLPClassifier

#### 10. Construct and train a neural network  (1 mark)

- Create an `MLPClassifier` which
    - has **1 hidden layer of 50 neurons** 
    - has **no regularization term**
    - trains for a maximum of **100 epochs** 
    - uses a batch size of **32**
- Fit the classifier using the standard `.fit()` member method.
- Plot the loss function value as a function of number of epochs (0.5 of mark).
  You can access the loss history through the `.loss_curve_` attribute of the `MLPClassifier` instance. 

In [ ]:
# create model
clf = MLPClassifier(max_iter=100, hidden_layer_sizes=(50,), batch_size=32, early_stopping=False)

# fit the data
clf.fit(x_train, y_train) 

In [ ]:
# define and plot loss data
loss_data = clf.loss_curve_

plt.plot(loss_data)
plt.xlabel('no. of epochs')
plt.ylabel('loss')
plt.show()

#### 11. Performance evaluation (0.5 mark)

- Using the testing dataset: 
    - Compute the overall accuracy for the classifier using the `MLPClassifier`'s `.score()` member method for both testing and training datasets.
    - Compute the confusion matrix (normalised in true labels), and plot it 
- Discuss the results

In [ ]:
# compute predicted values for both sets
y_pred_test = clf.predict(x_test)
y_pred_train = clf.predict(x_train)

In [ ]:
print('Accuracy for testing dataset: ' + str(clf.score(x_test, y_test)))
print('Accuracy for training dataset: ' + str(clf.score(x_train, y_train)))

In [ ]:
# compute the confusion matrix
clf_matrix = confusion_matrix(y_pred_test, y_test)

In [ ]:
### normalise the matrix
row_sums = clf_matrix.sum(axis=1)
new_matrix = clf_matrix / row_sums[:, np.newaxis]

### plot matrix
sns.heatmap(new_matrix, cmap='Blues', annot=True, xticklabels=wtype, yticklabels=wtype,fmt=".4f", square=True)
plt.ylabel('True Weather')
plt.xlabel('Predicted Weather')
plt.show()

The accuracy for the training dataset is larger by a very small amount so once could say that the model is slighly overfitting. The model learned rules that are slightly more specific to the training set. This means that some rules may not generalise very well beyond the training set.

The most accurately predicted class is Precipitation (accurately predicted 78% of the time) followed by Clear and Cloudy which is only predicted accurately 58% of the time. It could have been slightly overtrained in the training set with respect to clear and precipitation and not able to generalise for cloudy weather so well.

In [ ]:
# sanity check that accuracies are correct:

def accuracy(confusion_matrix):
   diagonal_sum = confusion_matrix.trace()
   sum_of_all_elements = confusion_matrix.sum()
   return diagonal_sum / sum_of_all_elements

print("Accuracy for training dataset: " + str(accuracy(clf_matrix)))

## 4. Neural networks in `Keras` (2 marks)
---
This section covers **2** exercises on constructing and training neural networks using the `Keras` library. `scikit-learn` is very easy to use, but libraries like `Keras` provide a lot more flexibility, which is why we will be using these extensively in the last two units of the _'Data science tools and machine learning'_ track.

In [ ]:
# Relevant import(s) for this section
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense

#### 12. Construct a neural network in `Keras` (1 mark)

- Create a `keras.Model` using the **Keras functional API**. The network should have:
    - An input layer with the same number of nodes as the number of features in `X`.
    - A single, densely connected hidden layer with **50 nodes** equipped with **ReLU activation**.
    - A densely connected output layer with **3 nodes** (the number of types of weather we're classifying) equipped with **softmax activation**.
- Compile the model the using the **Adam optimiser**, add `'accuracy'` as metric, and use either:
    - `categorical_crossentropy` loss, if you have one-hot encoded the targets `y`, or
    - `sparse_categorical_crossentropy` loss if you hare using integer-valued targets.
- Use the `.summary()` member method to print an overview of the model you have created, explain the output.

In [ ]:
### build model
in_1 = Input(shape=(23,))
dense_1 = Dense(50, activation='relu')(in_1)
out_1 = Dense(3, activation='softmax')(dense_1)
mdl = Model(in_1, out_1)

#compile
mdl.compile('adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
mdl.summary()

The output indicates that the total number of free (trainable) parameters is 1353. This comes from the structure of the neural network, i.e. $(23 \times 50) + 50 + (50 \times 3) + 3)$. This is also equal to the number of trainable parameters since there are no non-trainable parameters (the number of weights that are not updated during training with backpropagation).

#### 13. Train a `Keras` neural network (1 mark)

- Use the `.fit()` member method to train the network on the **training dataset** for **100 epochs** with a **batch size of 32**. Use **20% of the data for validation** and make sure to have `Keras` **shuffle** the training data between epochs. Save the fit history by doing `history_mld = .....`
- Print the classification accuracy using the `.evaluate()` member method, for both the training and testing dataset. Comment on the results.
- Plot val_loss and loss functions from the fit history. On the same plot, plot the sklearn curve from the excercise above. Note the sklearn NN does not provide a complementary validation loss history, so only plot the training loss.
- Comment on the results of the overall accuracy compared to the scikit-learn method.

In [ ]:
history_mld = mdl.fit(x_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=1)

In [ ]:
testing_acc = mdl.evaluate(x_test, y_test, verbose=1)
training_acc = mdl.evaluate(x_train, y_train, verbose=1)

In [ ]:
print("Accuracy for testing dataset: " + str(testing_acc[1]))
print("Accuracy for training dataset: " + str(training_acc[1]))

The training accuracy is slightly greater than the testing accuracy, as with the model above. However, the scikit-learn method yields better overall accuracy than this model which means that it is able to generalise better. The difference in accuracies is very small though so both models perform almost equally well, with a slight overfit.

In [ ]:
plt.plot(history_mld.history['val_loss'], label = "Validation Loss")
plt.plot(history_mld.history['loss'], label = "Loss")
plt.plot(loss_data, label='Training Loss')
plt.legend()
plt.show()

The training loss indicates how well the model is fitting the training data, while the validation loss indicates how well the model fits new data.

The graph shows overfitting; the model has too much capacity/flexibility. A model shows low loss on training, but high loss on testing datasets. The errors have high variance. The model therefore learns random structure in dataset that doesn't represent a generalisation: high variance, overfittting. In the above example, training loss is minimised, while validation loss saturates.

## 5. Regularisation (1.5 marks)
---
This section covers **2** exercises on the impact of weight regularisaton. Note that $L_{1}$- and $L_{2}$-regularisation may also be applied to the activation of intermediate layers. Also, a similar regularising effect could be achieved using **dropout** regularisation, which you are encouraged to try out, but which we won't study in this CP exercise.

In [ ]:
# Relevant import(s) for this section
from tensorflow.python.keras.regularizers import l1_l2

#### 14. Define `Keras` model factory method (0.5 mark)

- Define a python function called `big_model_fn` which takes the followng three arguments:
    - `l1`: A float specifying the $L_{1}$ regularisation factor (default value: 0)
    - `l2`: A float specifying the $L_{2}$ regularisation factor (default value: 0)
    - `name`: A string, specifying the name of the model (default value: None)
- Indside the function, you should:
    - Construct a `Keras` model using the functional API, which has:
        - An input layer with the same number of nodes as the number of features in `X`.
        - **Two** densely connected hidden layer with **100 nodes** each, both equipped with **ReLU activation**.
        - Both hidden layers should be subject to kernel regularisation (_i.e._ weight regularisation) with the regularisation factors specified as an input.
        - A densely connected output layer with **3 nodes** (the number of types of weather we're classifying) equipped with **softmax activation**.
        - A name given by the corresponding argument.
    - Compile the model in the same way as in **Exercise 12.**
- The function should return the compiled `Keras` model. 

The method will provide a convenient way of constructing and compiling a number of "big"/deep `Keras` models which differ only by their regularisation and name.

In [ ]:
def big_model_fn(l1=0, l2=0, name=None):
    in_2 = Input(shape=(X.shape[1],))
    regs = tf.keras.regularizers.l1_l2(l1=l1, l2=l2)
    dense_2 = Dense(100, activation='relu', kernel_regularizer=regs)(in_2)
    dense_3 = Dense(100, activation='relu', kernel_regularizer=regs)(dense_2)
    out_2 = Dense(3, activation='softmax')(dense_3)
    mod_reg = Model(in_2, out_2)
    mod_reg._name = name
    mod_reg.compile('adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return mod_reg
big_model_fn().summary()

#### 15. Train "big" models with and without regularisation (1 mark)

- Construct three "big" model using the factory method:
     - One with default parameters
     - One with `l1=0.0003` and  `name='Big model (L1-regularised)'`
     - One with `l2=0.003`  and `name='Big model (L2-regularised)'`
- Train each one as in **Exercise 13.**
- Compare first the loss history of the un-regularised "big" model to that of the small model from **Exercise 13**.
- Then, compare the loss histories of all three "big" models with that of the small model.
- Plot the loss and val loss of all 4 models and discuss the results. Target these points:
    - Compare the performance of deep vs shallow models on the testing sets
    - Compare the level of ovetraining (training vs testing loss)
    - Note: Don't be alarmed if the shallow network performs slightly better that the deeper ones, this is dataset dependant.

In [ ]:
### Model with default parameters
big_model_fn0 = big_model_fn(name='undegularised')
big_model_fn0.summary()
history_mld0 = big_model_fn0.fit(x_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

In [ ]:
plt.plot(history_mld.history['loss'], label = "small model")
plt.plot(history_mld0.history['loss'], label = "big unreg. model")
plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xlim(0,100)
plt.title('loss history of unregularised deep model against shallow model')
plt.show()

In [ ]:
big_model_fn1 = big_model_fn(l1=0.0003, name='BigModelL1Regularised')
history_mld1 = big_model_fn1.fit(x_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

In [ ]:
plt.plot(history_mld.history['loss'], label = "small model")
plt.plot(history_mld1.history['loss'], label = "l1 reg. model")
plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xlim(0,100)
plt.title('loss history of L1-regularised deep model against shallow model')
plt.show()

In [ ]:
big_model_fn2 = big_model_fn(l2=0.003, name='BigModelL2Regularised')
history_mld2 = big_model_fn2.fit(x_train, y_train, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, verbose=0)

In [ ]:
plt.plot(history_mld.history['loss'], label = "small model")
plt.plot(history_mld2.history['loss'], label = "l2 reg. model")
plt.legend()
plt.ylabel('loss')
plt.xlabel('epoch')
plt.xlim(0,100)
plt.title('loss history of L2-regularised deep model against shallow model')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(history_mld.history['loss'], label = "small model")
plt.plot(history_mld0.history['loss'], label = "big undeg. model")
plt.plot(history_mld1.history['loss'], label = "l1 reg. model")
plt.plot(history_mld2.history['loss'], label = "l2 reg. model")
plt.legend()
plt.title('Loss history of all models')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.xlim(0,100)
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history_mld.history['val_loss'], label = "small model")
plt.plot(history_mld0.history['val_loss'], label = "big undeg. model")
plt.plot(history_mld1.history['val_loss'], label = "l1 reg. model")
plt.plot(history_mld2.history['val_loss'], label = "l2 reg. model")
plt.legend()
plt.title('Validation loss history of all models')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.xlim(0,100)
plt.show()

## Testing loss

In [ ]:
print('Shallow model: ' + str(mdl.evaluate(x_test, y_test, verbose=1)))
print('Deep unregulated model: ' + str(big_model_fn0.evaluate(x_test, y_test, verbose=1)))
print('L1 regulated model: ' + str(big_model_fn1.evaluate(x_test, y_test, verbose=1)))
print('L2 regulated model: ' + str(big_model_fn2.evaluate(x_test, y_test, verbose=1)))

## Training loss

In [ ]:
print('Shallow model: ' + str(mdl.evaluate(x_train, y_train, verbose=1)))
print('Deep unregulated model: ' + str(big_model_fn0.evaluate(x_train, y_train, verbose=1)))
print('L1 regulated model: ' + str(big_model_fn1.evaluate(x_train, y_train, verbose=1)))
print('L2 regulated model: ' + str(big_model_fn2.evaluate(x_train, y_train, verbose=1)))

It looks like testing loss is greater than training loss, for all models except the deep undegulated network. This means that all models except the deep unregulated one are slightly overfitted, although the difference between losses is not major. For the unregulated, deep network training loss is significantly smaller than testing loss so we have underfitting.

Overall, it looks like the best-performing model is the L2-regulated network as the difference between testing and training loss is the smallest.

## 6. Bonus: Hyperparameter optimisation (1\*bonus\* mark)
---

This section covers **1** exercise on the on hyperparameter optimisation. 

In [ ]:
# Relevant import(s) for this section
from sklearn.model_selection import GridSearchCV, cross_val_score

---
_**Comment on simplified hyperparameter optimisation example:**_ You will try to perform a simple optimisation using a grid search

For convenience, we will be using the `scikit-learn` `MLPClassifier` as our base class, but the same principles apply to just about any ML model constructed in any framework. Just as in the examples in the lecture, we will restrict the hyperparameter space to just two dimensions:

* the number of hidden layers, `nb_layers`, and
* the number of nodes per hidden layer, `nb_nodes_per_layer`, which is taken to be the same for all hidden layers for simplicity.

Since the `scikit-learn` neural network classifier class doesn't support these two hyperparameters by default, provided is a simple wrapper class, that works exactly like `MLPClassifier`, it just takes the two parameters above as arguments in the constructor. Don't worry about understanding it in detail. This allows us to call 

In [ ]:
class MLPClassifierWrapper(MLPClassifier):
    """
    Wrapper around `sklearn.neural_network.MLPClassifier` with a convenient set 
    of properties (nb_layers and nb_nodes_per_layer) suitable for hyperparameter 
    optimisation exercises.
    
    Arguments:
        nb_layers: Integer, number of hidden layers
        nb_nodes_per_layer: Number of nodes per hidden layer, taken to be the 
            same for all for convenience.
    """

    def __init__ (self, nb_layers=1, nb_nodes_per_layer=100, **kwargs):
        
        # Member variables
        self._nb_layers = nb_layers
        self._nb_nodes_per_layer = nb_nodes_per_layer  
        
        # Call base class (`MLPClassifier`) constructor
        super(MLPClassifierWrapper, self).__init__(**kwargs)
        
        # Trigger `_set_architecture`
        self._set_architecture()
        return

    @property
    def nb_layers(self):
        return self._nb_layers
    
    @property
    def nb_nodes_per_layer(self):
        return self._nb_nodes_per_layer 

    @nb_layers.setter
    def nb_layers(self, value):
        self._nb_layers = value
        self._set_architecture()
        return
    
    @nb_nodes_per_layer.setter
    def nb_nodes_per_layer(self, value):
        self._nb_nodes_per_layer = value
        self._set_architecture()
        return
    
    def _set_architecture (self):
        """
        Sets the `hidden_layer_sizes` parameter of the base `MLPClassifier` 
        class, based on the two custom parameters we have chosen.
        """
        
        self.hidden_layer_sizes = tuple([self._nb_nodes_per_layer for _ in range(self._nb_layers)])
        return
    pass

---

#### 16. Perform a grid search (1 mark)

- Construct a python `dict` called `param_grid` which specifies the hyperparameter configurations to try for each parameter dimension. That is, it should have
    - `"nb_layers"` and `"nb_nodes_per_layer"` as keys, and
    - lists of integers as values, corresponding to the values of each parameter you want to try out (_e.g._ [1, 2, ...])
- Choose a reasonable set of values for each parameter; about a handful for each.
- Use the `GridSearchCV` class to perform _**3**_**-fold** cross validation (CV) optimisation of the validation **accuracy**
    - Hint: You can use the `n_jobs=...` argument to enable multi-processing, thereby speeding up the optimisation, at the expense of reproducibility.
- The base classifier should be an instance of `MLPClassifierWrapper` set to train for **100 epochs**.
- Present the results:
    - Print the best parameter configuration found. GridSearchCV has a public member which stores this. Read doc.
    - Print the mean and standard deviation of the test scores for the best configuration found. (_Hint:_ These can be found in the `.cv_results_` attribute)
- Discuss the results. What would happen if the best result is found on the edge of the parameter grid?

In [ ]:
# Create a dictionary of the hyperparameter configurations
param_grid = {"nb_layers" : [1, 2, 3, 4], 
              "nb_nodes_per_layer" : [50, 100, 150, 200]}

base_class = MLPClassifierWrapper(nb_layers=2, nb_nodes_per_layer=100)
base_class.fit(x_train, y_train)
base_class.score(x_train, y_train)

In [ ]:
# perform 3-fold cv optimisation of the validation accuracy
grid = GridSearchCV(estimator=base_class, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)

grid_results = grid.fit(x_train, y_train)

# Summarize the results in a readable format
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))

means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

The grid search indicates that the best combination of parameters is 1 hidden layer with 200 nodes per layer. Best attainable accuracy for the model is 69%. Hence, the greates number of layers and nodes is not always the optimum configuration.

If the best result is found on the edge of the parameter grid, when grid-search goes to fit the model with the edge,  the algorithm will complain that a point must not be less than or equal to the edge value, so we would get an error.